In [0]:
from google import colab
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import matplotlib.pyplot as plt
import csv
from collections import defaultdict, namedtuple
import os
import shutil
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix
import torch
import torchvision

In [3]:
# Download and unpack the training set and the test set

! wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip -P data
! wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip -P data
! wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip -P data
! wget https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/download/
! unzip data/GTSRB_Final_Training_Images.zip -d data
! unzip data/GTSRB_Final_Test_Images.zip -d data
! unzip data/GTSRB_Final_Test_GT.zip -d data

# Move the test set to data/test

! mkdir data/test
! mv data/GTSRB/Final_Test/Images/*.ppm data/test

# Download class names
! wget https://raw.githubusercontent.com/georgesung/traffic_sign_classification_german/master/signnames.csv -P data

Streaming output truncated to the last 5000 lines.
  inflating: data/GTSRB/Final_Test/Images/07646.ppm  
  inflating: data/GTSRB/Final_Test/Images/07647.ppm  
  inflating: data/GTSRB/Final_Test/Images/07648.ppm  
  inflating: data/GTSRB/Final_Test/Images/07649.ppm  
  inflating: data/GTSRB/Final_Test/Images/07650.ppm  
  inflating: data/GTSRB/Final_Test/Images/07651.ppm  
  inflating: data/GTSRB/Final_Test/Images/07652.ppm  
  inflating: data/GTSRB/Final_Test/Images/07653.ppm  
  inflating: data/GTSRB/Final_Test/Images/07654.ppm  
  inflating: data/GTSRB/Final_Test/Images/07655.ppm  
  inflating: data/GTSRB/Final_Test/Images/07656.ppm  
  inflating: data/GTSRB/Final_Test/Images/07657.ppm  
  inflating: data/GTSRB/Final_Test/Images/07658.ppm  
  inflating: data/GTSRB/Final_Test/Images/07659.ppm  
  inflating: data/GTSRB/Final_Test/Images/07660.ppm  
  inflating: data/GTSRB/Final_Test/Images/07661.ppm  
  inflating: data/GTSRB/Final_Test/Images/07662.ppm  
  inflating: data/GTSRB/Final_T

In [14]:
! wget https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/download

--2020-02-23 18:34:21--  https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/download
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?returnUrl=%2Fmeowmeowmeowmeowmeow%2Fgtsrb-german-traffic-sign%2Fdata [following]
--2020-02-23 18:34:21--  https://www.kaggle.com/account/login?returnUrl=%2Fmeowmeowmeowmeowmeow%2Fgtsrb-german-traffic-sign%2Fdata
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download’

download                [ <=>                ]   8.81K  --.-KB/s    in 0s      

2020-02-23 18:34:21 (69.4 MB/s) - ‘download’ saved [9026]



In [0]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [0]:
dataset = datasets.ImageFolder(
    '/content/data/GTSRB/Final_Training/Images',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [0]:
train_size = int(0.85 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [7]:
len(train_dataset)
len(test_dataset)

5882

In [0]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
   test_dataset,
  batch_size=64,
 shuffle=True,
  num_workers=0
  )

In [0]:
import numpy as np
def imshow(img):
  #npimg = img.to(device)
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [0]:
classes = (  'Speed limit (20km/h)',  'Speed limit (30km/h)',  'Speed limit (50km/h)',  'Speed limit (60km/h)',  'Speed limit (70km/h)',  'Speed limit (80km/h)',  'End of speed limit (80km/h)',  'Speed limit (100km/h)',  'Speed limit (120km/h)',  'No passing',  'No passing for vechiles over 3.5 metric tons',  'Right-of-way at the next intersection',  'Priority road',  'Yield',  'Stop',  'No vechiles',  'Vechiles over 3.5 metric tons prohibited',  'No entry',  'General caution', 'Dangerous curve to the left',  'Dangerous curve to the right',  'Double curve',  'Bumpy road',  'Slippery road',  'Road narrows on the right',  'Road work',  'Traffic signals',  'Pedestrians',  'Children crossing',  'Bicycles crossing',  'Beware of ice/snow',  'Wild animals crossing',  'End of all speed and passing limits',  'Turn right ahead',  'Turn left ahead',  'Ahead only',  'Go straight or right',  'Go straight or left',  'Keep right',  'Keep left',  'Roundabout mandatory',  'End of no passing',  'End of no passing by vechiles over 3.5 metric tons')

In [15]:
model = models.alexnet(pretrained=True)
# ct=0
# for child in model.children():
#   ct+=1
#   if(ct<7):
#     for param in child.parameters():
#       param.requires_grad = False

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [0]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 43) # 43  here refers to the number of outputs (class labels)
#model.fc = torch.nn.Linear(2048, 43)

In [0]:
device = torch.device('cuda')
model = model.to(device)

In [25]:
NUM_EPOCHS = 10
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.717103
1: 0.636008
2: 0.795478
3: 0.814689
4: 0.671030
5: 0.801938
6: 0.910745
7: 0.866712
8: 0.846481
9: 0.810949


In [0]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(x):
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x[None, ...]
    return x

In [0]:
x= cv2.imread('00005.png')
x=cv2.resize(x,(224,224))

In [0]:
x = preprocess(x)
x=x.to(device)
y = model(x)
y = F.softmax(y, dim=1)
y = y.flatten()

In [49]:
y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0.], device='cuda:0', grad_fn=<ViewBackward>)